# Βασικό Παράδειγμα AutoGen

Σε αυτό το δείγμα κώδικα, θα χρησιμοποιήσετε το [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework για να δημιουργήσετε έναν βασικό πράκτορα.

Ο στόχος αυτού του δείγματος είναι να σας δείξει τα βήματα που θα χρησιμοποιήσουμε αργότερα στα επιπλέον δείγματα κώδικα κατά την υλοποίηση των διαφορετικών μοτίβων πρακτόρων.


## Εισαγωγή των Απαραίτητων Πακέτων Python


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Δημιουργία του Πελάτη

Σε αυτό το παράδειγμα, θα χρησιμοποιήσουμε τα [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) για πρόσβαση στο LLM.

Το `model` ορίζεται ως `gpt-4o-mini`. Δοκιμάστε να αλλάξετε το μοντέλο σε κάποιο άλλο διαθέσιμο στην αγορά του GitHub Models για να δείτε διαφορετικά αποτελέσματα.

Ως μια γρήγορη δοκιμή, θα εκτελέσουμε απλώς μια απλή ερώτηση - `Ποια είναι η πρωτεύουσα της Γαλλίας`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Ορισμός του Πράκτορα

Τώρα που έχουμε ρυθμίσει τον `client` και επιβεβαιώσαμε ότι λειτουργεί, ας δημιουργήσουμε έναν `AssistantAgent`. Κάθε πράκτορας μπορεί να έχει:  
**name** - Ένα σύντομο όνομα που θα είναι χρήσιμο για την αναφορά του σε ροές πολλαπλών πρακτόρων.  
**model_client** - Ο client που δημιουργήσατε στο προηγούμενο βήμα.  
**tools** - Διαθέσιμα εργαλεία που ο Πράκτορας μπορεί να χρησιμοποιήσει για να ολοκληρώσει μια εργασία.  
**system_message** - Το metaprompt που ορίζει την εργασία, τη συμπεριφορά και τον τόνο του LLM.  

Μπορείτε να αλλάξετε το system message για να δείτε πώς ανταποκρίνεται το LLM. Θα καλύψουμε τα `tools` στο Μάθημα #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Εκτέλεση του Πράκτορα

Η παρακάτω συνάρτηση θα εκτελέσει τον πράκτορα. Χρησιμοποιούμε τη μέθοδο `on_message` για να ενημερώσουμε την κατάσταση του Πράκτορα με το νέο μήνυμα.

Σε αυτή την περίπτωση, ενημερώνουμε την κατάσταση με ένα νέο μήνυμα από τον χρήστη, το οποίο είναι `"Σχεδίασέ μου μια υπέροχη ηλιόλουστη διακοπή"`.

Μπορείτε να αλλάξετε το περιεχόμενο του μηνύματος για να δείτε πώς ανταποκρίνεται διαφορετικά το LLM.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
